## Introduction to TensorFlow
* All code available at https://github.com/zffchen78/tutorial
* Full set of instructions for convenience
```bash
virtualenv --system-site-packages tensorflow
source tensorflow/bin/activate
pip install -U jupyter
pip install -U "Choose the version according to your system here"
jupyter notebook # will open in browser
# run 'deactivate' to exit the virtualenv
```

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Hello World in TensorFlow

* Import the TensorFlow library
```python
import tensorflow as tf
```

* Create a session with TensorFlow's run time
```python
with tf.Session('') as sess:
```

* Create some operations
```python
  c = tf.constant('Hello, world!')
```

* Run the operations
```python
print(sess.run(c))
```

In [3]:
import tensorflow as tf
with tf.Session('') as sess:
    c = tf.constant('Hello, world!')
    print(sess.run(c))

Hello, world!


### InteractiveSession
* `with tf.Session(''):` is cumbersome in a REPL, for instance.
* Use `tf.InteractiveSession()`
```python
sess = tf.InteractiveSession()
```

In [4]:
sess = tf.InteractiveSession()

### Constants
```python
# float vector
a = tf.constant([1.0, 2.0, 3.0, 4.0])
# int32 vector
b = tf.constant([5, 6, 7, 8])
# int32 2x2 matrix
c = tf.constant([5, 6, 7, 8], shape=[2, 2])
# int32 2x2 matrix
d = tf.constant([[5, 6], [7, 8]])
# int32 2x2x1 matrix
e = tf.constant([[[5], [6]], [[7], [8]]])
```
### Types
* Integer types
  - tf.int32, tf.int64, ...
* Floating point types
  - tf.float32, tf.float64, tf.complex64, ..
* Other
  - tf.bool, tf.string

### Constants from numpy arrays

In [6]:
c = tf.constant(random.rand(2, 2, 2))
print(sess.run(c))

[[[ 0.24103464  0.21411158]
  [ 0.03174632  0.53075607]]

 [[ 0.83037872  0.39287798]
  [ 0.22634572  0.95331118]]]


### Expressions

In [7]:
x = tf.constant([[5, 6], [7, 8]])
z = tf.matmul(x, x) + tf.matmul(x, [[1, 0], [0, 1]])
# eval() method on expressions is a shorthand for sess.run
print(z.eval())  # sess.run(z)

[[ 72  84]
 [ 98 114]]


![Graph](data/expression.png)

### Common math functions
* Unary element-wise
```python
  tf.exp, tf.log, tf.neg, ...
```
* Binary element-wise
```python
  tf.add, tf.sub, tf.mul, tf.div, ...
```

In [7]:
tf.exp(1.23).eval()

3.4212296

In [73]:
print(tf.add(1, 2).eval())

# Broadcasting

# 2x3 matrix + 3 vector (1x3 matrix)
print(tf.add([[1., 2, 3], [4, 5, 6]], [10, 20, 30]).eval())

# 2x3 matrix + 2x1 matrix
print(tf.add([[1., 2, 3], [4, 5, 6]], [[10], [20]]).eval())

3
[[ 11.  22.  33.]
 [ 14.  25.  36.]]
[[ 11.  12.  13.]
 [ 24.  25.  26.]]


  * Reduction
  ```python
  tf.reduce_sum, tf.reduce_mean, ...
  ```

In [33]:
x = tf.constant([[1., 2, 3], [4, 5, 6]])
print(x.eval())
print(tf.reduce_sum(x, reduction_indices=[0]).eval())
print(tf.reduce_sum(x, reduction_indices=[1]).eval())
print(tf.reduce_sum(x).eval())

[[ 1.  2.  3.]
 [ 4.  5.  6.]]
[ 5.  7.  9.]
[  6.  15.]
21.0


* Rest of TensorFlow built as libraries on top of these primitives
* `tf.nn` library
  - Neural network related functions - `softmax`, `relu`, `conv2d`
* `tf.image` library
  - Image processing functions - `crop`, `adjust_brightness`
* `tf.train` - Neural net training related functions
  - `GradientDescentOptimizer`, `exponential_decay`

E.g.,
$$(y_0, y_1, ..., y_n) = softmax(x_0, x_1, ..., x_n)$$

$$ y_i = \frac{e^{x_i}}{\sum_i{e^{x_j}}}$$

In [56]:
logits = tf.constant([[-1., 0, 1], [1, 2, 3], [10, 20, 30]])
print(logits.eval())
print(tf.nn.softmax(logits).eval())

[[ -1.   0.   1.]
 [  1.   2.   3.]
 [ 10.  20.  30.]]
[[  9.00305733e-02   2.44728476e-01   6.65240943e-01]
 [  9.00305733e-02   2.44728476e-01   6.65240943e-01]
 [  2.06105999e-09   4.53978682e-05   9.99954581e-01]]


In [60]:
labels = [0, 2, 1]
print(tf.one_hot(labels, 3, 1., 0.).eval())

[[ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]]


In [64]:
qlogp = - tf.log(tf.nn.softmax(logits)) * tf.one_hot(labels, 3, 1., 0.)
print(qlogp.eval())
per_example_loss = tf.reduce_sum(qlogp, [1])
print(per_example_loss.eval())
loss = tf.reduce_mean(per_example_loss)
print(loss.eval())

[[  2.40760589   0.           0.        ]
 [  0.           0.           0.40760601]
 [  0.          10.00004578   0.        ]]
[  2.40760589   0.40760601  10.00004578]
4.27175


In [65]:
per_example_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels)
print(loss.eval())

4.27175


### Variables
* Primary mechanism for "State" in TensorFlow

In [8]:
# Create a variable with some initial value
v = tf.Variable([[1.0, 2.0], [3.0, 5.0]])

In [9]:
# Get an operation that when run, initializes all variables
# declared so far
init = tf.initialize_all_variables()

In [10]:
# Initialize the variables
init.run()
# Inspect the variables
print(v.eval())

[[ 1.  2.]
 [ 3.  5.]]


In [17]:
# Assign a new value to the variable
assign_v = v.assign(random.rand(2, 2))

In [18]:
# Perform the assignment and inspect the new value
sess.run(assign_v)
print(v.eval())

[[ 0.28583112  0.93428499]
 [ 0.33675203  0.75247449]]


### Gradient
  * Automatic symbolic differentiation. 
  * Very useful for gradient-base learning algorithm.

In [72]:
x = tf.placeholder(tf.float32)
y = x * x * x + 3 * x + 4  # y = x^3 + 3x + 4
dy, = tf.gradients(y, x)    # dy/dx = 3 * x^2 + 3

print(dy.eval(feed_dict={x:[-1, 0, 1, 2]}))

[  6.   3.   6.  15.]
